In [88]:
import os
os.environ['OPENAI_API_KEY'] = 'YOUR API KEY
import openai
import string
import numpy as np
import pandas as pd
from gpt_index import (
    GPTKeywordTableIndex, 
    SimpleDirectoryReader, 
    LLMPredictor,
    PromptHelper,
    GPTSimpleVectorIndex,
    GPTListIndex,
    readers
)
from langchain import OpenAI

In [10]:
path2file = r'C:\Users\aa\Desktop\University\Semester 5\DSSG\DRK\scraped.csv'
df = pd.read_csv(path2file)

In [20]:
df_texts = df['text']

In [26]:
i=1
for text in df_texts:
    filepath = os.path.join(r'C:\Users\aa\Desktop\University\Semester 5\DSSG\DRK\text_files',f'{i}.txt')
    with open(filepath, 'w',encoding="utf-8") as f:
        f.write(text)
        i += 1

In [81]:
def construct_index(directory_path):
    # set maximum input size
    max_input_size = 4096
    # set number of output tokens (default)
    num_outputs = 256
    # maximum chunk overlap
    chunk_overlap = 20 

    # define LLM
    llm_predictor = LLMPredictor(llm=OpenAI(temperature=0.5, model_name='text-davinci-002', max_tokens=num_outputs))
    prompt_helper = PromptHelper(max_input_size, num_outputs, chunk_overlap, chunk_size_limit=512)
    
    documents = SimpleDirectoryReader(directory_path).load_data()

    index = GPTSimpleVectorIndex(
        documents, llm_predictor=llm_predictor, prompt_helper=prompt_helper
    )

    index.save_to_disk('index1.json')

    return index

In [84]:
# text_files_test has 30 files for indexing
path_files = r'C:\Users\aa\Desktop\University\Semester 5\DSSG\DRK\text_files_test'
index = construct_index(path_files)

> Adding chunk: Sie mÃ¶chten die Kita GÃ¤nseblÃ¼mchen in Neusta...
> Adding chunk: Wir bieten vielfÃ¤ltige, auf die BedÃ¼rfnisse u...
> Adding chunk: z.B. durch KindertagesstÃ¤tten, ChÃ¶re und Musi...
> Adding chunk: Hier finden Sie Dokumente rund um das Thema Pfl...
> Adding chunk: Finden Sie hier die Standorte unserer Kleiderco...
> Adding chunk: Mit Ihrer Spende leisten Sie einen wertvollen B...
> Adding chunk: Zum Stadtfest in PÃ¶ÃŸneck prÃ¤sentierte sich d...
> Adding chunk: Sie selbst, Ihr Partner, ein Elternteil oder ei...
> Adding chunk: Wir freuen uns, dass Sie sich fÃ¼r unsere Kinde...
> Adding chunk: Als GeschÃ¤ftsmann war Henry Dunant 1859 in Ita...
> Adding chunk: Er gestattete den Ã¶sterreichischen Ã„rzten an ...
> Adding chunk: Am vergangenen Wochenende sind die Mitarbeiteri...
> Adding chunk: VerÃ¤nderungen der politischen Gegebenheiten in...
> Adding chunk: die sich aus den Schadenereignissen ergeben kÃ¶...
> Adding chunk: ausgebildetes Fachpersonal mit entsprechender 

In [85]:
index.load_from_disk('index1.json')

In [89]:
def ask_texts():
    RETRIEVE_DOCS = 0
    SUMMARIZE_DOCS = 1

    state = RETRIEVE_DOCS

    while True:
        if (state == RETRIEVE_DOCS):
            query = input("Welche Art von Dokumente wollen Sie? ")
            
            response = index.query(query, similarity_top_k=20, verbose=True, response_mode='default')
            retrieved_docs = []
            for sn in response.source_nodes:
                retrieved_docs.append(readers.Document(sn.source_text))
                
            state = SUMMARIZE_DOCS
            
            index2 = GPTListIndex(retrieved_docs)
            
            while state == SUMMARIZE_DOCS:
                query = input("Was würden Sie gerne dazu wissen wollen? ")
                
                if(query == 'exit'):
                    state = RETRIEVE_DOCS
                    break
                
                response = index2.query(query, response_mode="compact", verbose=True)
                print(response.response)

In [ ]:
ask_texts()

Welche Art von Dokumente wollen Sie? Kannst du mir Dokumente zur Kältehilfe für Obdachlose holen
> Top 20 nodes:
> [Node 4a26feeb-04b4-4081-9423-238fb0db39a9] [Similarity score:                     0.829564] Hier finden Sie Dokumente rund um das Thema Pflege und Heimaufnahme.

Sie kÃ¶nnen jederzeit eine ...
> [Node 164c18b7-f0e9-4d73-93db-5c1669916919] [Similarity score:                     0.815038] Sie mÃ¶chten die Kita GÃ¤nseblÃ¼mchen in Neustadt an der Orla direkt mit einer Spende unterstÃ¼tz...
> [Node c024efb5-15a3-40e1-b551-5d2bb611abd4] [Similarity score:                     0.812976] In den SoziallÃ¤den in Schleiz und PÃ¶ÃŸneck unterstÃ¼tzen wir bedÃ¼rftige Menschen mit gÃ¼nstige...
> [Node 2d546b90-2d08-46a7-b334-815ca996477e] [Similarity score:                     0.810647] Infos fÃ¼r Sie kostenfrei 

rund um die Uhr...
> [Node 3920d32e-5db6-4a7d-97cd-189436f6ba90] [Similarity score:                     0.803632] Zum Stadtfest in PÃ¶ÃŸneck prÃ¤sentierte sich das DRK mit dem 

> Refined response: 

Yes, the locations of the clothing containers can be found here. In addition, we provide counseling and guidance to residents according to their individual needs, offer accompanying help, create a supportive environment, and act in the best interests of our residents. This allows residents to gain, maintain, and regain skills which contribute to their overall wellbeing.
> Refine context: Der medizinische Einsatzleiter, Herr Hartmut Ja...
> Refined response: 

Yes, the locations of the clothing containers can be found here. In addition, we provide counseling and guidance to residents according to their individual needs, offer accompanying help, create a supportive environment, and act in the best interests of our residents. This allows residents to gain, maintain, and regain skills which contribute to their overall wellbeing.
> [query] Total LLM token usage: 7298 tokens
> [query] Total embedding token usage: 27 tokens
> Adding chunk: Hier finden Sie Dokumente rund 

Was würden Sie gerne dazu wissen wollen? Was bietet DRK bremen an
> Searching in chunk: Hier finden Sie Dokumente rund um das Thema Pfl...
> Searching in chunk: Sie mÃ¶chten die Kita GÃ¤nseblÃ¼mchen in Neusta...
> Searching in chunk: In den SoziallÃ¤den in Schleiz und PÃ¶ÃŸneck un...
> Searching in chunk: Infos fÃ¼r Sie kostenfrei 

rund um die Uhr...
> Searching in chunk: Zum Stadtfest in PÃ¶ÃŸneck prÃ¤sentierte sich d...
> Searching in chunk: Sie selbst, Ihr Partner, ein Elternteil oder ei...
> Searching in chunk: Mit Ihrer Spende leisten Sie einen wertvollen B...
> Searching in chunk: Schirmen Sie das Kind vor neugierigen Blicken a...
> Searching in chunk: FÃ¼r die Beantragung Ihrer Mutter-/Vater-Kind-K...
> Searching in chunk: Wir freuen uns, dass Sie sich fÃ¼r unsere Kinde...
> Searching in chunk: Am vergangenen Wochenende sind die Mitarbeiteri...
> Searching in chunk: Wir freuen uns, dass Sie sich fÃ¼r unser Angebo...
> Searching in chunk: Wir sind auch in Corona-Zeiten fÃ¼r Sie 